<a href="https://colab.research.google.com/github/mourya716/FMML_LAB_23B21A4562/blob/main/FMML_M1L2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Machine Learning terms and metrics

FMML Module 1, Lab 2

In this lab, we will show a part of the ML pipeline by using the California Housing dataset. There are 20640 samples, each with 8 attributes like income of the block, age of the houses per district etc. The task is to predict the cost of the houses per district. We will use the scikit-learn library to load the data and perform some basic data preprocessing and model training. We will also show how to evaluate the model using some common metrics, split the data into training and testing sets, and use cross-validation to get a better estimate of the model's performance.

In [ ]:
import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

rng = np.random.default_rng(seed=42)

In [ ]:
dataset = datasets.fetch_california_housing()
# Dataset description
print(dataset.DESCR)

.. _california_housing_dataset:

California Housing dataset
--------------------------

**Data Set Characteristics:**

    :Number of Instances: 20640

    :Number of Attributes: 8 numeric, predictive attributes and the target

    :Attribute Information:
        - MedInc        median income in block group
        - HouseAge      median house age in block group
        - AveRooms      average number of rooms per household
        - AveBedrms     average number of bedrooms per household
        - Population    block group population
        - AveOccup      average number of household members
        - Latitude      block group latitude
        - Longitude     block group longitude

    :Missing Attribute Values: None

This dataset was obtained from the StatLib repository.
https://www.dcc.fc.up.pt/~ltorgo/Regression/cal_housing.html

The target variable is the median house value for California districts,
expressed in hundreds of thousands of dollars ($100,000).

This dataset was derived

Given below are the list of target values. These correspond to the house value derived considering all the 8 input features and are continuous values. We should use regression models to predict these values but we will start with a simple classification model for the sake of simplicity. We need to just round off the values to the nearest integer and use a classification model to predict the house value.

In [ ]:
print("Orignal target values:", dataset.target)

dataset.target = dataset.target.astype(int)

print("Target values after conversion:", dataset.target)
print("Input variables shape:", dataset.data.shape)
print("Output variables shape:", dataset.target.shape)

Orignal target values: [4.526 3.585 3.521 ... 0.923 0.847 0.894]
Target values after conversion: [4 3 3 ... 0 0 0]
Input variables shape: (20640, 8)
Output variables shape: (20640,)


The simplest model to use for classification is the K-Nearest Neighbors model. We will use this model to predict the house value with a K value of 1. We will also use the accuracy metric to evaluate the model.

In [ ]:
def NN1(traindata, trainlabel, query):
    """
    This function takes in the training data, training labels and a query point
    and returns the predicted label for the query point using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    query: numpy array of shape (d,) where d is the number of features

    returns: the predicted label for the query point which is the label of the training data which is closest to the query point
    """
    diff = (
        traindata - query
    )  # find the difference between features. Numpy automatically takes care of the size here
    sq = diff * diff  # square the differences
    dist = sq.sum(1)  # add up the squares
    label = trainlabel[np.argmin(dist)]
    return label


def NN(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the nearest neighbour algorithm

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is the label of the training data which is closest to each test point
    """
    predlabel = np.array([NN1(traindata, trainlabel, i) for i in testdata])
    return predlabel

We will also define a 'random classifier', which randomly allots labels to each sample

In [ ]:
def RandomClassifier(traindata, trainlabel, testdata):
    """
    This function takes in the training data, training labels and test data
    and returns the predicted labels for the test data using the random classifier algorithm

    In reality, we don't need these arguments but we are passing them to keep the function signature consistent with other classifiers

    traindata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    trainlabel: numpy array of shape (n,) where n is the number of samples
    testdata: numpy array of shape (m,d) where m is the number of test samples and d is the number of features

    returns: the predicted labels for the test data which is a random label from the training data
    """

    classes = np.unique(trainlabel)
    rints = rng.integers(low=0, high=len(classes), size=len(testdata))
    predlabel = classes[rints]
    return predlabel

We need a metric to evaluate the performance of the model. Let us define a metric 'Accuracy' to see how good our learning algorithm is. Accuracy is the ratio of the number of correctly classified samples to the total number of samples. The higher the accuracy, the better the algorithm. We will use the accuracy metric to evaluate and compate the performance of the K-Nearest Neighbors model and the random classifier.

In [ ]:
def Accuracy(gtlabel, predlabel):
    """
    This function takes in the ground-truth labels and predicted labels
    and returns the accuracy of the classifier

    gtlabel: numpy array of shape (n,) where n is the number of samples
    predlabel: numpy array of shape (n,) where n is the number of samples

    returns: the accuracy of the classifier which is the number of correct predictions divided by the total number of predictions
    """
    assert len(gtlabel) == len(
        predlabel
    ), "Length of the ground-truth labels and predicted labels should be the same"
    correct = (
        gtlabel == predlabel
    ).sum()  # count the number of times the groundtruth label is equal to the predicted label.
    return correct / len(gtlabel)

Let us make a function to split the dataset with the desired probability. We will use this function to split the dataset into training and testing sets. We will use the training set to train the model and the testing set to evaluate the model.

In [ ]:
def split(data, label, percent):
    # generate a random number for each sample
    rnd = rng.random(len(label))
    split1 = rnd < percent
    split2 = rnd >= percent

    split1data = data[split1, :]
    split1label = label[split1]
    split2data = data[split2, :]
    split2label = label[split2]
    return split1data, split1label, split2data, split2label

We will reserve 20% of our dataset as the test set. We will not change this portion throughout our experiments

In [ ]:
testdata, testlabel, alltraindata, alltrainlabel = split(
    dataset.data, dataset.target, 20 / 100
)
print("Number of test samples:", len(testlabel))
print("Number of train samples:", len(alltrainlabel))
print("Percent of test data:", len(testlabel) * 100 / len(dataset.target), "%")

Number of test samples: 4144
Number of train samples: 16496
Percent of test data: 20.07751937984496 %


## Experiments with splits

Let us reserve some of our train data as a validation set

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)

What is the accuracy of our classifiers on the train dataset?

In [ ]:
trainpred = NN(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using nearest neighbour algorithm:", trainAccuracy*100, "%")

trainpred = RandomClassifier(traindata, trainlabel, traindata)
trainAccuracy = Accuracy(trainlabel, trainpred)
print("Training accuracy using random classifier: ", trainAccuracy*100, "%")

Training accuracy using nearest neighbour algorithm: 100.0 %
Training accuracy using random classifier:  16.4375808538163 %


For nearest neighbour, the train accuracy is always 1. The accuracy of the random classifier is close to 1/(number of classes) which is 0.1666 in our case. This is because the random classifier randomly assigns a label to each sample and the probability of assigning the correct label is 1/(number of classes). Let us predict the labels for our validation set and get the accuracy. This accuracy is a good estimate of the accuracy of our model on unseen data.

In [ ]:
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")


valpred = RandomClassifier(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using random classifier:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.10852713178294 %
Validation accuracy using random classifier: 16.884689922480618 %


Validation accuracy of nearest neighbour is considerably less than its train accuracy while the validation accuracy of random classifier is the same. However, the validation accuracy of nearest neighbour is twice that of the random classifier. Now let us try another random split and check the validation accuracy. We will see that the validation accuracy changes with the split. This is because the validation set is small and the accuracy is highly dependent on the samples in the validation set. We can get a better estimate of the accuracy by using cross-validation.

In [ ]:
traindata, trainlabel, valdata, vallabel = split(
    alltraindata, alltrainlabel, 75 / 100)
valpred = NN(traindata, trainlabel, valdata)
valAccuracy = Accuracy(vallabel, valpred)
print("Validation accuracy using nearest neighbour algorithm:", valAccuracy*100, "%")

Validation accuracy using nearest neighbour algorithm: 34.048257372654156 %


You can run the above cell multiple times to try with different random splits.
We notice that the accuracy is different for each run, but close together.

Now let us compare it with the accuracy we get on the test dataset.

In [ ]:
testpred = NN(alltraindata, alltrainlabel, testdata)
testAccuracy = Accuracy(testlabel, testpred)

print("Test accuracy:", testAccuracy*100, "%")

Test accuracy: 34.91795366795367 %


### Try it out for yourself and answer:
1. How is the accuracy of the validation set affected if we increase the percentage of validation set? What happens when we reduce it?
2. How does the size of the train and validation set affect how well we can predict the accuracy on the test set using the validation set?
3. What do you think is a good percentage to reserve for the validation set so that thest two factors are balanced?

Answer for both nearest neighbour and random classifier. You can note down the values for your experiments and plot a graph using  <a href=https://matplotlib.org/stable/gallery/lines_bars_and_markers/step_demo.html#sphx-glr-gallery-lines-bars-and-markers-step-demo-py>plt.plot<href>. Check also for extreme values for splits, like 99.9% or 0.1%

Answers:-

1. Effect of Validation Set Size on Accuracy

For Nearest Neighbour and Random Classifier:

Increasing the Percentage of Validation Set:

Nearest Neighbour:

More data in the validation set means fewer data points in the training set. This can lead to underfitting, as the model may not capture complex patterns with limited training data. Validation accuracy may initially increase (better representation of generalization), but it will eventually decrease as the training data becomes insufficient.

For large validation sets (e.g., 99.9%), the model is almost not learning from the data, leading to poor accuracy.


Random Classifier:

Accuracy remains around random guessing (e.g., 50% for binary classification), irrespective of validation set size. The classifier does not learn from data, so changing the split doesn't impact the performance meaningfully.



Decreasing the Percentage of Validation Set:

Nearest Neighbour:

A smaller validation set provides less reliable estimates of generalization performance. The model may show high variance in validation accuracy. Training accuracy may be high, but it might not generalize well to unseen data.


Random Classifier:

The effect remains similar to the above scenario; accuracy is close to random guessing.




2. Effect of Train and Validation Set Size on Prediction of Test Set Accuracy

Nearest Neighbour:

With a balanced train and validation split (e.g., 80% train, 20% validation), the validation set gives a good approximation of test set performance.

If the validation set is too small, it might not represent the entire data distribution, leading to inaccurate predictions of test set accuracy.

If the training set is too small, the model may not generalize well, and the validation set won't reflect true model performance.


Random Classifier:

The validation set is not useful for predicting test set accuracy because the classifier does not depend on training data. The accuracy remains around the expected value for random guessing.



3. Good Percentage for the Validation Set

A good balance is typically 20% validation set and 80% training set. This ensures that:

Nearest Neighbour:

Enough data to train and validate the model, minimizing overfitting and providing a reasonable estimate of generalization.


Random Classifier:

Irrelevant for learning since it's based on random guessing. The percentage split doesn’t affect performance.




Experimentation and Graph

To better visualize the effects, we can experiment with different splits and plot the results using plt.plot. Here is a simple code snippet using matplotlib to show how to set up the plot:

import matplotlib.pyplot as plt

# Hypothetical accuracies for different validation set percentages
validation_percentages = [0.1, 1, 10, 20, 50, 80, 99.9]
nearest_neighbour_accuracies = [0.4, 0.45, 0.55, 0.60, 0.58, 0.45, 0.20]  # Example values
random_classifier_accuracies = [0.5] * len(validation_percentages)  # Stays around 0.5

# Plotting
plt.figure(figsize=(10, 5))
plt.plot(validation_percentages, nearest_neighbour_accuracies, label='Nearest Neighbour')
plt.plot(validation_percentages, random_classifier_accuracies, label='Random Classifier', linestyle='--')
plt.xlabel('Validation Set Percentage')
plt.ylabel('Accuracy')
plt.title('Effect of Validation Set Size on Accuracy')
plt.legend()
plt.xscale('log')  # Optional: Use a logarithmic scale for better visualization
plt.show()

This code shows how accuracy varies with different validation set sizes. Experimenting with extreme values like 99.9% or 0.1% will demonstrate the significant impact on the Nearest Neighbour classifier compared to the random classifier, which remains unaffected.

> Exercise: Try to implement a 3 nearest neighbour classifier and compare the accuracy of the 1 nearest neighbour classifier and the 3 nearest neighbour classifier on the test dataset. You can use the KNeighborsClassifier class from the scikit-learn library to implement the K-Nearest Neighbors model. You can set the number of neighbors using the n_neighbors parameter. You can also use the accuracy_score function from the scikit-learn library to calculate the accuracy of the model.

## Multiple Splits

One way to get more accurate estimates for the test accuracy is by using <b>cross-validation</b>. Here, we will try a simple version, where we do multiple train/val splits and take the average of validation accuracies as the test accuracy estimation. Here is a function for doing this. Note that this function will take a long time to execute. You can reduce the number of splits to make it faster.

In [ ]:
def AverageAccuracy(alldata, alllabel, splitpercent, iterations, classifier=NN):
    """
    This function takes in the data, labels, split percentage, number of iterations and classifier function
    and returns the average accuracy of the classifier

    alldata: numpy array of shape (n,d) where n is the number of samples and d is the number of features
    alllabel: numpy array of shape (n,) where n is the number of samples
    splitpercent: float which is the percentage of data to be used for training
    iterations: int which is the number of iterations to run the classifier
    classifier: function which is the classifier function to be used

    returns: the average accuracy of the classifier
    """
    accuracy = 0
    for ii in range(iterations):
        traindata, trainlabel, valdata, vallabel = split(
            alldata, alllabel, splitpercent
        )
        valpred = classifier(traindata, trainlabel, valdata)
        accuracy += Accuracy(vallabel, valpred)
    return accuracy / iterations  # average of all accuracies

In [ ]:
avg_acc = AverageAccuracy(alltraindata, alltrainlabel, 75 / 100, 10, classifier=NN)
print("Average validation accuracy:", avg_acc*100, "%")
testpred = NN(alltraindata, alltrainlabel, testdata)

print("Test accuracy:", Accuracy(testlabel, testpred)*100, "%")

Average validation accuracy: 33.58463539517022 %
Test accuracy: 34.91795366795367 %


This is a very simple way of doing cross-validation. There are many well-known algorithms for cross-validation, like k-fold cross-validation, leave-one-out etc. This will be covered in detail in a later module. For more information about cross-validation, check <a href=https://en.wikipedia.org/wiki/Cross-validation_(statistics)>Cross-validatioin (Wikipedia)</a>

### Questions
1. Does averaging the validation accuracy across multiple splits give more consistent results?
2. Does it give more accurate estimate of test accuracy?
3. What is the effect of the number of iterations on the estimate? Do we get a better estimate with higher iterations?
4. Consider the results you got for the previous questions. Can we deal with a very small train dataset or validation dataset by increasing the iterations?

Answers:-
1. Averaging Validation Accuracy Across Multiple Splits

Consistency: Yes, averaging validation accuracy across multiple splits tends to provide more consistent results. This process reduces variance, as it smooths out the impact of any particular split that may be unrepresentative of the overall data distribution.


2. Estimate of Test Accuracy

Accuracy of Estimate: Averaging over multiple splits offers a more accurate estimate of test accuracy. It captures a broader view of model performance across different data partitions, mitigating the influence of anomalies in any single split.


3. Effect of Number of Iterations on Estimate

Impact of Iterations: Increasing the number of iterations (or folds in cross-validation) generally leads to a better estimate of model performance. More iterations allow for more robust averaging of accuracy, further reducing variance and increasing reliability.


4. Dealing with Small Train or Validation Dataset by Increasing Iterations

Limitations: While increasing iterations can help in stabilizing estimates, it does not compensate for the inherent limitations of having a very small training or validation dataset. A small training set may not provide enough information for the model to learn effectively, while a small validation set may not reliably reflect the generalization performance.

Recommendation: In cases of very small datasets, it's often better to consider techniques like data augmentation, transfer learning, or acquiring more data rather than solely relying on increasing iterations.

> Exercise: How does the accuracy of the 3 nearest neighbour classifier change with the number of splits? How is it affected by the split size? Compare the results with the 1 nearest neighbour classifier.

Answers:-
1. Accuracy of 3 Nearest Neighbour Classifier with Number of Splits

Number of Splits:

For k-fold cross-validation, the accuracy estimate tends to stabilize as the number of splits increases. However, the specific accuracy achieved will depend on the model's capability to generalize across diverse splits.

3 Nearest Neighbour (3-NN) tends to perform more robustly with higher splits as it averages the influence of the nearest neighbors, reducing sensitivity to individual data points.


Impact:

More Splits (e.g., 10-fold cross-validation): The accuracy estimate becomes more stable, with less variance across different splits.

Fewer Splits (e.g., 2-fold cross-validation): The model might show higher variance in accuracy, as each split significantly changes the train/validation sets, leading to inconsistent results.



2. Effect of Split Size on Accuracy

3 Nearest Neighbour:

Larger Validation Set (smaller training set): The accuracy can decrease because the model has less training data to capture the structure of the data. 3-NN may not find well-defined clusters due to limited training samples.

Smaller Validation Set (larger training set): Accuracy generally improves as the model benefits from more training data. The prediction is more robust, as 3-NN can leverage more information from the training set to find the nearest neighbors effectively.


Comparison to 1 Nearest Neighbour (1-NN):

1-NN Classifier: It is highly sensitive to noise and outliers since it only considers the single closest point. With small training sets, it may overfit and show high variance in accuracy.

3-NN Classifier: More robust to outliers compared to 1-NN as it averages the decision based on three nearest neighbors. This makes it less sensitive to changes in training set size or noise, showing more consistent accuracy.



3. Comparison Between 3-NN and 1-NN with Split Size

1-NN Classifier:

Highly Variable: Small changes in the training set significantly impact predictions, leading to a large variance in accuracy. A large training set is crucial to ensure reliable predictions.

Effect of Split Size: As the validation set size increases, accuracy might drastically change due to the high sensitivity to individual points.


3-NN Classifier:

More Stable: The 3-NN model averages its decisions, reducing the sensitivity to changes in the training set.

Effect of Split Size: The accuracy is more stable and consistent compared to 1-NN. It benefits more from larger training sets and shows a gradual decline in accuracy as the training set size decreases, without the sharp drops observed in 1-NN.



Summary

1-NN: More prone to overfitting, high variance, very sensitive to training set size.

3-NN: More stable, less sensitive to individual data points, performs better with varying training set sizes.


Experimental Analysis

To see the differences in practice, you can use k-fold cross-validation with different split sizes for both 1-NN and 3-NN. Then plot their accuracy against the number of folds or the size of the validation set. Here's a simple outline:

import matplotlib.pyplot as plt
from sklearn.model_selection import cross_val_score
from sklearn.neighbors import KNeighborsClassifier
from sklearn.datasets import make_classification

# Generating a synthetic dataset
X, y = make_classification(n_samples=500, n_features=10, n_informative=5, n_classes=2, random_state=42)

# Splits and results storage
splits = [2, 5, 10, 20